In [273]:
import numpy
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [274]:
V_coarse = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
], dtype=float)

E_coarse = numpy.array([
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 0],
])

m = V_coarse.shape[0]
n = 3
alphas = numpy.linspace(0, 1, n + 2)[1:-1]
V_fine = numpy.vstack(
    list(V_coarse) + 
    [(v1 - v0) * alpha + v0 for v0, v1 in V_coarse[E_coarse] for alpha in alphas]
)

E_fine = []
for i in range(m):
    for j in range(n + 1):
        E_fine.append([
            i if j == 0 else (m - 1) + n * i + j, 
            ((i + 1) % m) if j == n else (m - 1) + n * i + j + 1])
E_fine = numpy.array(E_fine)

coarse_row_order = E_coarse[:, 0].flatten().tolist() + [0]
fine_row_order = E_fine[:, 0].flatten().tolist() + [0]

U_coarse = numpy.zeros(V_coarse.shape)
U_coarse[V_coarse[:, 0] > 0.999, 1] = 1
U_fine = numpy.zeros(V_fine.shape)
U_fine[V_fine[:, 0] > 0.999, 1] = 1

In [275]:
coarse_coll = False
if coarse_coll:
    V_coll, V_fem = V_coarse, V_fine
    E_coll, E_fem = E_coarse, E_fine
    U_coll, U_fem = U_coarse, U_fine
    coll_row_order, fem_row_order = coarse_row_order, fine_row_order
else:
    V_coll, V_fem = V_fine, V_coarse
    E_coll, E_fem = E_fine, E_coarse
    U_coll, U_fem = U_fine, U_coarse
    coll_row_order, fem_row_order = fine_row_order, coarse_row_order

V_fem = V_coll
E_fem = E_coll
U_fem = U_coll
fem_row_order = coll_row_order

In [276]:
def upsample(V, E, edge_splits):
    alphas = numpy.linspace(0, 1, edge_splits + 2)[1:-1]
    upsampled_V = [V]
    downsample_buddy = [[[i, 1]] for i in range(V.shape[0])]
    i= V.shape[0]
    for e in E:
        v0, v1 = V[e]
        for alpha in alphas:
            downsample_buddy[e[0]].append([i, 1-alpha])
            downsample_buddy[e[1]].append([i, alpha])
            upsampled_V.append((v1 - v0) * alpha + v0)
            i += 1
    return numpy.vstack(upsampled_V), downsample_buddy

In [277]:
def point_on_edge(p, e0, e1):
    d = numpy.linalg.norm(numpy.cross(e0 - p, e1 - p))**2 / numpy.linalg.norm(e1 - e0)**2
    return d < 1e-8

def point_edge_coord(p, e0, e1):
    e = e1 - e0
    return (p - e0).dot(e) / numpy.linalg.norm(e)**2

def compute_W(V_coll, V_fem, E_fem):
    W = numpy.zeros((V_coll.shape[0], V_fem.shape[0]))

    for i, v in enumerate(V_coll):
        for e in E_fem:
            e0, e1 = V_fem[e]
            t = point_edge_coord(v, e0, e1)
            if point_on_edge(v, e0, e1) and 0 <= t <= 1:
                W[i, e[0]] = 1 - t
                W[i, e[1]] = t
                break

    assert(numpy.linalg.norm(V_coll - W @ V_fem) < 1e-12)
    return W

In [278]:
def downsample(W, downsample_buddies):
    W_downsampled = numpy.vstack([
        # W[i] + W[buddies].sum(axis=0)
        sum(weight * W[buddy] for buddy, weight in buddies)
        for buddies in downsample_buddies])
    W_downsampled /= W_downsampled.sum(axis=1)[:, None]
    return W_downsampled

In [288]:
edge_splits = 10
V_coll_upsampled, downsample_buddy = upsample(V_coll, E_coll, edge_splits)
W = compute_W(V_coll_upsampled, V_fem, E_fem)
W_new = downsample(W, downsample_buddy)

In [289]:
X_coll = V_coll + W_new @ U_fem
X_fem = V_fem + U_fem

fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], mode="markers+lines", name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_coll_upsampled[:, 0], y=V_coll_upsampled[:, 1], mode="markers", name="V_coll_upsampled"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], mode="markers+lines", name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], mode="markers+lines", name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], mode="markers+lines", name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800, title="Displace V_fem")

In [281]:
X_coll = V_coll + U_coll
X_fem = V_fem + W_new.T @ U_coll
X_coll2 = V_coll + W_new @ (X_fem - V_fem)

fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], mode="markers+lines", name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_coll_upsampled[:, 0], y=V_coll_upsampled[:, 1], mode="markers", name="upsampled V_coll"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], mode="markers+lines", name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], mode="markers+lines", name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_coll2[coll_row_order, 0], y=X_coll2[coll_row_order, 1], mode="markers+lines", name="Mapped Deformed V_coll"), row=2, col=2),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], mode="markers+lines", name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800, title="Displace V_coll")